In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_image='D:/Documents/Capstone/Project -2/Dataset/zinc_Vs_non-Zinc/train'

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.1,
                                vertical_flip=False)

In [4]:
training_set=train_datagen.flow_from_directory(train_image,
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)


Found 600 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 zoom_range=0.2)

test_set=train_datagen.flow_from_directory('D:/Documents/Capstone/Project -2/Dataset/zinc_Vs_non-Zinc/test',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 60 images belonging to 2 classes.


In [6]:
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(224,224,3)
                     )
 
base_model.trainable=False
 
model = tf.keras.Sequential([
        base_model,  
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

In [7]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 batch_normalization_203 (Ba  (None, 5, 5, 1536)       10752     
 tchNormalization)                                               
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                        

In [9]:
early = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)

In [10]:
batch_size=32
STEP_SIZE_TRAIN = training_set.n//training_set.batch_size
STEP_SIZE_VALID = test_set.n//test_set.batch_size

# fit model
history = model.fit(training_set, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=[early])

Epoch 1/10
120/120 [==============================] - 172s 1s/step - loss: 0.6797 - accuracy: 0.7150 - val_loss: 0.5557 - val_accuracy: 0.8000
Epoch 2/10
120/120 [==============================] - 159s 1s/step - loss: 0.5007 - accuracy: 0.8300 - val_loss: 0.3315 - val_accuracy: 0.9000
Epoch 3/10
120/120 [==============================] - 152s 1s/step - loss: 0.4130 - accuracy: 0.8800 - val_loss: 0.6015 - val_accuracy: 0.8667
Epoch 4/10
120/120 [==============================] - 149s 1s/step - loss: 0.3133 - accuracy: 0.9083 - val_loss: 0.1656 - val_accuracy: 0.9500
Epoch 5/10
120/120 [==============================] - 158s 1s/step - loss: 0.2129 - accuracy: 0.9450 - val_loss: 0.1847 - val_accuracy: 0.9333
Epoch 6/10
120/120 [==============================] - 157s 1s/step - loss: 0.1527 - accuracy: 0.9450 - val_loss: 0.0767 - val_accuracy: 0.9833
Epoch 7/10
120/120 [==============================] - 159s 1s/step - loss: 0.1260 - accuracy: 0.9517 - val_loss: 0.0759 - val_accuracy: 0.9833